In [22]:
## Import necessary modules
import os,sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, FormatStrFormatter, ScalarFormatter
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee

sys.path.append(os.path.abspath('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/lib/'))
from ce_funclib import determine_kernzeit as dtkz

%matplotlib tk

## Deklarationen

In [23]:
arcpath='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/datenhalde/1458/' #Leseverzeichnis
neededcols=[0,1,2,3,5,12,21] # Liste der Spalten, die aus dem Excelfile gelesen werden sollen
new_colnames=['tstp','clls','ange','verb','tt','acw','lost'] # Namen, die spaeter auf die Spalten kommen
shitcols=('Verbindungszeit [hh:mm:ss]','Nachbearbeitungszeit [hh:mm:ss]')
converts={shitcols[0]:str,shitcols[1]:str}

## Funktionen

In [24]:
######## GET A LIST OF MATCHING .xls FILES FROM THE GIVEN DIRECTORY
def collectxlfiles(arcpath):
    xlfilelist=list()

    for xlfile in os.listdir(arcpath):
        if xlfile.startswith('1458_daily'):
            xlfileabs=os.path.join(arcpath,xlfile)
            xlfilelist.append(xlfileabs)
    return sorted(xlfilelist)

In [25]:
def turn_xls_to_df(file):

    excel_df=pd.read_excel(file,skiprows=3,skip_footer=1,usecols=neededcols,converters=converts)# die ersten 3 werden nicht benötigt, letzte auch nicht
    excel_df['Timestamp'] = pd.to_datetime(excel_df['Timestamp'], format=' %d.%m.%Y %H:%M ') # statt string soll das ein datetime werden
    excel_df.columns=new_colnames
    excel_df2=excel_df.set_index('tstp').copy() # die timestamps sollen der index sein
        
    return excel_df2

In [26]:
def add_averages(frame):

    frame['avtt']=(frame['tt']/frame['verb'])/60
    frame['avht']=(frame['ht']/frame['verb'])/60
    frame['avacw']=(frame['acw']/frame['verb'])/60
    colorder_with_abs=['clls','verb','lost','avht','avtt','avacw','ht','tt','acw']
    frame_with_abs=frame[colorder_with_abs].fillna(0)
    
    return frame_with_abs

In [53]:
def labelspucker(x_index):
    xx=x_index
    print(xx.month.unique())
    von_monat=xx.month.unique()[0]
    bis_monat=xx.month.unique()[-1]
    von_jahr=xx.year.unique()[0]
    bis_jahr=xx.year.unique()[-1]
    
    if (von_monat == bis_monat):
        labeltext=str(calendar.month_abbr[von_monat])+' '+str(von_jahr)
        
    return labeltext

## Datensammeln und Frame mit allen Daten herstellen

In [28]:
xlfilelist=collectxlfiles(arcpath)

In [29]:
greatframe=pd.DataFrame() # leeren df initialisieren
for i in xlfilelist:
    i_frame=turn_xls_to_df(i)
    greatframe=greatframe.append(i_frame) 
# alle files werden ordentlich in df konvertiert und an den ausserhalb der funktion kreierten df angehangen
# greatframe # hier sind alle daten vollständig enthalten, die gebraucht werden, der Ur-Frame

In [30]:
## Anrufzeiten statt datetime oder string nach timedelta (Sekunden) gewandelt
greatframe[['tt','acw']]=greatframe[['tt','acw']].apply(pd.to_timedelta).astype('timedelta64[s]')
greatframe['ht'] = (greatframe['tt']+greatframe['acw'])

In [31]:
## add kern- und nebenzeit column
greatframe['bz'] = greatframe.index.map(dtkz)

In [32]:
daily_frame=greatframe.groupby([pd.Grouper(freq='D'),pd.Grouper('bz')]).sum().copy() # Nebenzeit nach Tagen gruppiert
daily_frame=daily_frame.unstack('bz').fillna(0).stack('bz')

In [33]:
g = daily_frame.sum(level=['tstp']) # einen Zwischenframe erstellen, der nur die Kern- und Nebenzeit summiert
g.assign(bz='g').set_index('bz', append=True).sort_index() # die Summe dem alten Frame beigeben
daily=daily_frame.append(g.assign(bz='g').set_index('bz', append=True)).sort_index() # die Summe dem alten Frame beigeben

In [34]:
daily_av=add_averages(daily)  # durschnittliche Zeiten acw/calls etc.

## Slicing und Plotting

In [35]:
d_ix=daily_av.index.get_level_values(0)  # das ist der datetime-index (level 0), kann mit monat,jahr usw. bedient werden
apr_18=daily_av.loc[(d_ix.month == 4) & (d_ix.year == 2018)]
mar_18=daily_av.loc[(d_ix.month == 3) & (d_ix.year == 2018)]

In [36]:
apr_18.head()

clls  verb  lost      avht      avtt     avacw     ht     tt  \
tstp       bz                                                                 
2018-04-01 g    3.0   3.0   0.0  1.716667  1.155556  0.561111  309.0  208.0   
           k    0.0   0.0   0.0  0.000000  0.000000  0.000000    0.0    0.0   
           n    3.0   3.0   0.0  1.716667  1.155556  0.561111  309.0  208.0   
2018-04-02 g    3.0   2.0   1.0  3.516667  2.516667  1.000000  422.0  302.0   
           k    3.0   2.0   1.0  3.516667  2.516667  1.000000  422.0  302.0   

                 acw  
tstp       bz         
2018-04-01 g   101.0  
           k     0.0  
           n   101.0  
2018-04-02 g   120.0  
           k   120.0

In [68]:
def abs_plot(frame):
    gcol="#777777"
    lostcol="#AC003A"
    verbcol="#008EC4"
    ncol="#000000"
    kcol="#4F7DFF"

    sonn = WeekdayLocator(6)
    weekFormatter = DateFormatter('%a %b %d')

    fig,ax=plt.subplots()

    ax.xaxis.set_major_locator(sonn)
    ax.xaxis.set_major_formatter(weekFormatter)

    ax.grid(which='major', axis='y', linestyle=':', zorder=0)
    ax.set_axisbelow(True)


    d_ix=frame.index.get_level_values(0).unique()
    g_val=frame.xs('g', level='bz')['clls']
    k_val=frame.xs('k', level='bz')['clls']
    n_val=frame.xs('n', level='bz')['clls']
    verb=frame.xs('g', level='bz')['verb']
    lost=frame.xs('g', level='bz')['lost']

    bar_g=ax.bar(d_ix,g_val,color=gcol,width=0.8, label='calls')
    bar_n=ax.bar(d_ix,n_val, color=ncol, width=0.4, align='edge',label='davon nebenzeit')
    bar_verb=ax.bar(d_ix,verb,color=verbcol,width=-0.3, label='verbunden')
    bar_lost=ax.bar(d_ix,lost,color=lostcol,width=-0.3, bottom=verb,label='verloren')
    #bar_k=ax.bar(d_ix,k_val, color=kcol, width=0.2, align='center')


    ax.set_xlabel(labelspucker(d_ix))
    ax.set_ylabel('calls',rotation=90)
    
    
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels)

    
    
    
    
    fig.show()
    
    #return d_ix
    
    

In [69]:
plotyear=2018

for i in range(1,5):
    print(i)
    plotmonth=daily_av.loc[(d_ix.month == i) & (d_ix.year == plotyear)]
    abs_plot(plotmonth)

1
Int64Index([1], dtype='int64', name='tstp')
2
Int64Index([2], dtype='int64', name='tstp')
3
Int64Index([3], dtype='int64', name='tstp')
4
Int64Index([4], dtype='int64', name='tstp')


Int64Index([11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='tstp')